In [24]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import clickhouse_connect
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 20)

In [15]:
# you can change config here)))
tickers = ["AAPL.US", "TSLA.US", "BAC.US", "MSFT.US", "GOOGL.US", "AMZN.US", "V.US", "JNJ.US", "WMT.US", "NVDA.US", "META.US", "XOM.US", "UNH.US", "KO.US", "MA.US", "NFLX.US", "DIS.US", "SBUX.US", ]  # 18

names = ['Apple', 'Tesla Motors', 'Bank of America', 'Microsoft Corporation', 'Alphabet Inc. Class A', 'Amazon.com', 'Visa', 'Johnson and Johnson', 'Walmart Inc.', 'Nvidia', 'Meta Platforms Inc.', 'Exxon Mobil', 'UnitedHealth Group', 'Coca Cola', 'Mastercard', 'Netflix', 'Walt Disney Company', 'Starbucks']

timeframe_in = "D1"  # timeframe for neural network training - input and we will trade on the same timeframe

CLICKHOUSE_CLOUD_HOSTNAME = 'csi4142-clickhouse.mershab.xyz'
CLICKHOUSE_CLOUD_USER = 'csi'
CLICKHOUSE_CLOUD_PASSWORD = 'iceberg'


TABLE_NAME = 'company_dimension_table_test'

In [16]:
# Do it for all shares
df_list = []
single_ticker_df = {}
for ticker in tickers:
    temp_df  = pd.DataFrame(pd.read_csv(f"../data/company_dimension/{timeframe_in}/{ticker}_{timeframe_in}.csv", parse_dates=['datetime']))
    temp_df['ticker'] = ticker
    df_list.append(temp_df) 

df_list

[       datetime    open    high     low   close     volume   ticker
 0    1998-01-02    0.12    0.14    0.12    0.14  170539824  AAPL.US
 1    1998-01-05    0.14    0.14    0.13    0.14  152723900  AAPL.US
 2    1998-01-06    0.14    0.17    0.13    0.16  433041952  AAPL.US
 3    1998-01-07    0.16    0.16    0.15    0.15  251914152  AAPL.US
 4    1998-01-08    0.15    0.16    0.15    0.16  188994988  AAPL.US
 5    1998-01-09    0.16    0.17    0.15    0.16  214486244  AAPL.US
 6    1998-01-12    0.15    0.16    0.15    0.16  125714764  AAPL.US
 7    1998-01-13    0.16    0.17    0.16    0.17  152129936  AAPL.US
 8    1998-01-14    0.17    0.17    0.17    0.17  138278308  AAPL.US
 9    1998-01-15    0.16    0.17    0.16    0.16  130645452  AAPL.US
 ...         ...     ...     ...     ...     ...        ...      ...
 6629 2024-03-02  179.65  179.67  179.59  179.66    5186609  AAPL.US
 6630 2024-03-04  176.15  176.90  173.79  175.10   54836758  AAPL.US
 6631 2024-03-05  175.08  175.13  

In [17]:
consolidated_df = pd.concat(df_list, ignore_index=True)

In [18]:

consolidated_df['datetime'] = pd.to_datetime(consolidated_df['datetime'],unit='s')
consolidated_df['datetime'] = consolidated_df['datetime'].dt.strftime('%Y-%m-%d')
consolidated_df

,datetime,open,high,low,close,volume,ticker
0,1998-01-02,0.12,0.14,0.12,0.14,170539824,AAPL.US
1,1998-01-05,0.14,0.14,0.13,0.14,152723900,AAPL.US
2,1998-01-06,0.14,0.17,0.13,0.16,433041952,AAPL.US
3,1998-01-07,0.16,0.16,0.15,0.15,251914152,AAPL.US
4,1998-01-08,0.15,0.16,0.15,0.16,188994988,AAPL.US
5,1998-01-09,0.16,0.17,0.15,0.16,214486244,AAPL.US
6,1998-01-12,0.15,0.16,0.15,0.16,125714764,AAPL.US
7,1998-01-13,0.16,0.17,0.16,0.17,152129936,AAPL.US
8,1998-01-14,0.17,0.17,0.17,0.17,138278308,AAPL.US
9,1998-01-15,0.16,0.17,0.16,0.16,130645452,AAPL.US


In [19]:
client = clickhouse_connect.get_client(host=CLICKHOUSE_CLOUD_HOSTNAME, port=443, user=CLICKHOUSE_CLOUD_USER, password=CLICKHOUSE_CLOUD_PASSWORD,database='default')

# Check if table exists and create if it doesn't
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    datetime String,
    open Float64,
    high Float64,
    low Float64,
    close Float64,
    volume UInt32,
    ticker String
) ENGINE = MergeTree()
ORDER BY datetime
"""

#client.command(f'DROP TABLE IF EXISTS {TABLE_NAME}')
client.command(create_table_query)
print(client.ping())
print("connected to " + CLICKHOUSE_CLOUD_HOSTNAME + "\n")

True
connected to csi4142-clickhouse.mershab.xyz



In [20]:
print(consolidated_df.dtypes)

datetime     object
open        float64
high        float64
low         float64
close       float64
volume        int64
ticker       object
dtype: object


In [21]:
client.insert_df(TABLE_NAME, consolidated_df)

In [22]:

result_df = client.query(f'SELECT * FROM {TABLE_NAME}')

result_df.summary

{'read_rows': '0',
 'read_bytes': '0',
 'written_rows': '0',
 'written_bytes': '0',
 'total_rows_to_read': '0',
 'result_rows': '0',
 'result_bytes': '0',
 'elapsed_ns': '1406855'}